# <u> LSTM ENCODER DECODER

# Data

In [1]:
import utils
import numpy as np

#utils.setup_nltk()
PRESIDENT = 'obama'
speeches = utils.read_all_text_files(PRESIDENT)

In [2]:
assert np.mean([len(x) for x in speeches]) == 4878.9

### Preprocessing

In [3]:
filter_list = [':', '(', ')', ',', '-',]
filtered_speeches = []

for speech in speeches:
    filtered_speech = []
    for word in speech:
        # filter out unwanted words
        if word not in filter_list:
            # lower word
            filtered_speech.append(word.lower())
    filtered_speeches.append(filtered_speech)

### Create N-Grams

In [4]:
from nltk import ngrams

WINDOW = 5
grams = [ngrams(s, WINDOW+1) for s in filtered_speeches]
flat_grams = [ng for speech in grams for ng in speech]

### Split Data

In [5]:
import pandas as pd

X = [' '.join(list(x[0:WINDOW])) for x in flat_grams]
Y = [x[-1] for x in flat_grams]
df = pd.DataFrame.from_dict({'x':X, 'y':Y})

# persist
csv_name = '../data/lstm/' + PRESIDENT + '_preproc/encdec_' + str(WINDOW) + 'grams.csv'
df.to_csv(csv_name, index=False)

In [6]:
#assert len(df) == 230020

In [7]:
import torchtext

XFIELD = torchtext.data.Field(sequential=True)
YFIELD = torchtext.data.Field(sequential=True)
DATA = torchtext.data.TabularDataset(csv_name,'csv', 
                                     [('x', XFIELD),('y', YFIELD)], skip_header=True)

XFIELD.build_vocab(DATA)  
YFIELD.build_vocab(DATA)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes f

In [8]:
from torchtext.data import BucketIterator, Iterator
import torch

BATCH_SIZE = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator = Iterator(DATA, BATCH_SIZE, device=device, train=True)

/home/gabriel/.local/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [9]:
assert device.type == 'cuda'

# Neural Networks

### Encoder

In [10]:
import torch.nn as nn

class Encoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, bidirectional=True)

    def forward(self, x, h0, c0):
        x = self.embedding(x).unsqueeze(0)
        out, (h0, c0) = self.lstm(x, (h0, c0))
        return out, (h0, c0)

### Decoder

In [11]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers=num_layers, dropout=0.5, bidirectional=True)
        self.dense = nn.Linear(hidden_size*2, vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)
  
    def forward(self, x, h0, c0):
        x = self.embedding(x)
        x, (h0, c0) = self.lstm(x, (h0, c0))
        x = self.dense(x.squeeze(0))
        x = self.softmax(x)
        return x, (h0, c0)

### Attention

In [432]:
class AttentionDecoder(nn.Module):
    def __init__(self, vocab_size, hidden_size, embedding_dim, num_layers):
        super(AttentionDecoder, self).__init__()
        # attention
        self.lin = nn.Linear(hidden_size*2, 1)
        self.soft = nn.Softmax(dim=0)
        # decoding
        self.lstm = nn.LSTM(hidden_size*2, hidden_size, num_layers=num_layers, bidirectional=True)
        self.dense = nn.Linear(hidden_size*2, vocab_size)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, x, apply=False):
        # create attention weights
        lin = self.lin(x).squeeze()
        attn_weights = self.soft(lin)
        # [5, 64]
        
        # apply attention weights
        ord_att_weights = attn_weights.repeat_interleave(HIDDEN_SIZE*2).view(WINDOW,BATCH_SIZE,HIDDEN_SIZE*2)
        applied = ord_att_weights * x
        # [5, 64, 40]
        
        # decode
        lstm_out, (h0, c0) = self.lstm(applied)
        dense = self.dense(lstm_out)
        # [5, 64, 10582]
        soft = self.softmax(dense)
        
        # aggregate
        agg = torch.sum(soft, dim=0)
        # [64, 10582]
        
        return agg

### Training

In [438]:
HIDDEN_SIZE = 20
EMBEDDING_SIZE = 50
NUM_LAYERS = 2
LR = 0.01
ENC_LEARNING_RATE = LR
DEC_LEARNING_RATE = LR
criterion = nn.NLLLoss()

encoder = Encoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
decoder = Decoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device)
# decoder = AttentionDecoder(len(XFIELD.vocab), HIDDEN_SIZE, EMBEDDING_SIZE, NUM_LAYERS).to(device) # FOR ATTENTION
enc_optimizer = torch.optim.Adam(encoder.parameters(), lr = ENC_LEARNING_RATE)
dec_optimizer = torch.optim.Adam(decoder.parameters(), lr = DEC_LEARNING_RATE)

In [439]:
from tqdm import tqdm

EPOCHS = 1
for ep in range(EPOCHS):
    ep_loss = 0
    
    for batch in tqdm(train_iterator):
        if len(batch) != BATCH_SIZE: break;
        inp = batch.x
        target = batch.y
        
        # init
        loss = 0
        h0 = torch.zeros(NUM_LAYERS*2, BATCH_SIZE, HIDDEN_SIZE).to(device)
        c0 = torch.zeros(NUM_LAYERS*2, BATCH_SIZE, HIDDEN_SIZE).to(device)
        enc_optimizer.zero_grad()
        dec_optimizer.zero_grad()
        
        # encode
        # enc_outs = [] FOR ATTENTION
        for w in range(inp.size(0)):
            enc_out, (h0, c0) = encoder(inp[w], h0, c0)
            # enc_outs.append(enc_out.to(device)) # FOR ATTENTION
            
        # attention decoding
        # enc_outs = torch.cat(enc_outs).to(device) # FOR ATTENTION
        # dec_out = decoder(enc_outs) # FOR ATTENTION
        
        # decode
        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (h0, c0) = decoder(cur, h0, c0)        
        cur = torch.argmax(dec_out,dim=1)
        
        # loss
        # target_onehot = torch.nn.functional.one_hot(target.squeeze(), len(YFIELD.vocab))
        loss += criterion(dec_out, target.squeeze())
        
        # optimize
        ep_loss += loss
        loss.backward()
        enc_optimizer.step()
        dec_optimizer.step()
        
    print('AVG_LOSS={}, (ABS={})'.format(round((ep_loss/(len(DATA)/BATCH_SIZE)).item(),4), round(ep_loss.item(),2)))

100%|█████████▉| 3592/3593 [00:51<00:00, 70.28it/s]

AVG_LOSS=5.8373, (ABS=20970.0)


# Generate Text!

### functions

In [171]:
from torch import torch

def voc_index(words):
    return torch.tensor([XFIELD.vocab.stoi[x] for x in words]).to(device)

def predict(inp, RND_FACTOR=0, multiply=False):
    with torch.no_grad():

        h0 = torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        c0 = torch.zeros(2*NUM_LAYERS, BATCH_SIZE, HIDDEN_SIZE).to(device)
        
        for w in range(inp.size(0)):
                enc_out, (h0, c0) = encoder(inp[w], h0, c0)

        cur = inp[WINDOW-1].unsqueeze(0)
        dec_out, (h0, c0) = decoder(cur, h0, c0)
        
        # randomize
        if multiply:
            rnd = torch.rand(dec_out.shape).to(device) * RND_FACTOR + 1
            cur = torch.argmax(dec_out * rnd,dim=1)
        else:
            rnd = torch.rand(dec_out.shape).to(device) * RND_FACTOR
            cur = torch.argmax(dec_out.add(rnd),dim=1)

        return YFIELD.vocab.itos[cur[0].item()]

def generate(intro=['good', 'evening', 'ladies', 'and', 'gentlemen'], multiply=False, rnd_factor=10, length=100):
    text = intro
    for i in range(length):
        cur_window = text[-WINDOW:]
        vecs = voc_index(cur_window).view(WINDOW,1).repeat(1,BATCH_SIZE)
        text.append(predict(vecs, rnd_factor, multiply))

    return ' '.join(text)

### create N speeches

In [172]:
generated = []
GENERATE_N = 10
for i in range(GENERATE_N):
    generated.append(generate(intro=['good', 'evening', 'ladies', 'and', 'gentlemen'], 
                              multiply=True, 
                              rnd_factor=1, 
                              length=100))

generated[0]

'good evening ladies and gentlemen has a place that ’ s a lot of years of the world ’ s future . i ’ m not just a nation that has a nation that we ’ re also willing to make sure that i ’ m going to be a difference . we ’ ve sent the same purpose of our union . but i ’ m going to be a lot of americans . we will not be able to be a tax cut that they ’ re going to be a difference . and that ’ s why we ’ re going to'

# Evaluate

In [173]:
import metrics

# persist
for i in range(len(generated)):
    with open("../data/lstm/{}_generated/{}.txt".format(PRESIDENT, str(i)), "w") as text_file:
        text_file.write(generated[i]) 

# scores
mean_cos, std_cos, cos_sim = metrics.get_cosine_sim_tfidf("obama", "lstm/obama_generated", 0, print_results=True)
rouge = metrics.get_rouge_score("obama", "lstm/obama_generated", 0, print_results=True)

mean cosine similarity for generated speech 0: 0.03367195645989311
standard deviation of cosine similarity for generated speech 0: 0.02038927823655809
mean rouge score for generated speech 0: 0.046501440825484304
standard deviation of rouge score for generated speech 0: 0.024417678102751885


# Save Models